In [35]:
import pandas as pd
import openai
import os
from utils import get_val_scores, make_transcript
from AptTagger import AptTagger
from TruthLabeler import TruthLabeler

openai.api_key_path = "/home/alex/.personal/openAIkey"  # read protected to my 174 account

In [36]:
nt = 21
val_scores_df = get_val_scores(val_path=f"data/validation/{nt}_val_labels.txt")
val_df = pd.read_json(f"data/validation/{nt}_transcripts.json")
val_df.index = val_df["message_id"]
val_df = val_df.join(val_scores_df, how="inner")
main_df = val_df

# drop message bde7de09-80ef-44f5-bdce-48426cbae387 (imperatives), cf6936bb-cad2-4269-a83b-1a760a36e333 (brain size)
main_df = main_df.drop(["bde7de09-80ef-44f5-bdce-48426cbae387", "cf6936bb-cad2-4269-a83b-1a760a36e333"])

apt_tagger = AptTagger(model_name=None)  # regex

main_df["tag_mask"] = apt_tagger.tag(main_df["assistant_text"])
main_df["annotated_assistant_text"] = main_df.apply(lambda x: apt_tagger.annotate(x["assistant_text"], x["tag_mask"]), axis=1)
main_df["annotated_transcript"] = main_df.apply(make_transcript, axis=1, args=(True,))
print(len(main_df))
main_df.head()

19


,message_id,prompt,assistant_text,parent_id,prev_messages,role,model,template_name,gen_kwargs,subject,answer_index,correct_answer,random_incorrect_answer,random_other_subject,tag_mask,annotated_assistant_text,annotated_transcript,templated_annotated_transcript,scores
4584ad83-28df-4c51-8f9f-f533d0649ae0,4584ad83-28df-4c51-8f9f-f533d0649ae0,"<s>[INST] <<SYS>>\nYou are a helpful, respectf...",Hello! I'm here to help you with your question...,4dceb093b525b4687f5da7a758ed5b85,[Answer this question:\n\nWhat happens to the ...,assistant,meta-llama/Llama-2-7b-chat-hf,vanilla,max_length=4096;temperature=1.0;repetition_pen...,High School Physics,1,The image moves away from the mirror and gets ...,The image moves toward the mirror and gets tal...,Professional Accounting,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Hello[[1]]! I'm here to help you with your que...,USER: Answer this question:\n\nWhat happens to...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,"[N/A, N/A, False, Certainly True, Certainly Tr..."
f45dabef-0d27-4abc-b77b-8e02c2b1c6cc,f45dabef-0d27-4abc-b77b-8e02c2b1c6cc,"<s>[INST] <<SYS>>\nYou are a helpful, respectf...","Oh, wow! That's such an awesome question! *exc...",bf8608f71ce32d8d115cfbec5a6f56c9,"[Of the following animals, which has the longe...",assistant,meta-llama/Llama-2-7b-chat-hf,5yo,max_length=4096;temperature=1.0;repetition_pen...,Human Aging,1,Blue whale,Dog,High School Geography,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","Oh, wow[[1]]! That's such an awesome question[...","USER: Of the following animals, which has the ...",### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,"[N/A, N/A, N/A, N/A, N/A, N/A, Certainly True,..."
c20d1031-c292-4039-89f2-209ca862efc1,c20d1031-c292-4039-89f2-209ca862efc1,"<s>[INST] <<SYS>>\nYou are a helpful, respectf...",The best translation into predicate logic for ...,f142a6a4ac2472d72aa90b127c2f4ddc,[Select the best translation into predicate lo...,assistant,meta-llama/Llama-2-7b-chat-hf,answer_only,max_length=4096;temperature=1.0;repetition_pen...,Formal Logic,3,Nd ⊃ (Bj ∨ Sl),Dn ≡ (Bx • Ly),Professional Medicine,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",The best translation into predicate logic for ...,USER: Select the best translation into predica...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,"[N/A, Certainly False, N/A, Certainly False, C..."
94ea7b6a-dcbe-4fb4-a94c-5427447cc333,94ea7b6a-dcbe-4fb4-a94c-5427447cc333,"<s>[INST] <<SYS>>\nYou are a helpful, respectf...",Thank you for asking! I'm here to help you wit...,b5423fa1a663e0a13ff0770d371b681c,[Answer this question:\n\nCells taken from a b...,assistant,meta-llama/Llama-2-7b-chat-hf,vanilla,max_length=4096;temperature=1.0;repetition_pen...,Human Aging,1,Stop dividing when their telomere strands are ...,Never divide,Clinical Knowledge,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Thank you for asking[[1]]! I'm here to help yo...,USER: Answer this question:\n\nCells taken fro...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,"[N/A, N/A, N/A, Certainly True, Certainly True..."
395ef1bd-0918-4fc5-b1ea-818863f2f4e6,395ef1bd-0918-4fc5-b1ea-818863f2f4e6,"<s>[INST] <<SYS>>\nYou are a helpful, respectf...",My apologize for my previous response. Accordi...,bbd08ff74cf76a35212dda7a2366a8d4,[Which one of the following groups of patients...,assistant,meta-llama/Llama-2-7b-chat-hf,multiturn_debate_for_true,max_length=4096;temperature=1.0;repetition_pen...,Clinical Knowledge,3,Any patient who is acutely unwell pre-operativ...,Only patients over 60 years of age.,College Physics,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",My apologize for my previous response[[1]]. Ac...,USER: Which one of the following groups of pat...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,"[N/A, Certainly False, Certainly True, Certain..."


In [3]:
# annotated_val_df = pd.read_json("data/validation/12_transcripts.json")
# main_df = annotated_val_df

In [4]:
# from Retriever import Retriever
# retriever = Retriever(
#     use_IVF=True,
#     encoder_name="BAAI/bge-small-en",
#     pretrained_index_path='retrieval/20230601.en.wiki_embeddings_bge-small-en_IVF.index',
# )

In [37]:
truth_labeler = TruthLabeler(
                retriever=None,
                model_name="gpt-4",
                temperature=1,
                n_samples=5,
                uncertainty_bias=2,
                na_bias=0,
)

In [38]:
all_out_df = truth_labeler.label(
    ids=main_df["message_id"],
    annotated_transcripts=main_df["annotated_transcript"], 
    correct_answers=main_df["correct_answer"],
    retrieval_queries=main_df["assistant_text"],
    n_threads=2)

SKIPPING: scores must be one of ['N/A', 'Determinable - Certainly True', 'Determinable - True', 'Determinable - Uncertain', 'Determinable - False', 'Determinable - Certainly False'], but found ['N/A', 'N/A', 'Determinable - Uncertain', 'Determinable - Certainly True', 'Determinable - Certainlly True', 'Determinable - Certainly True']
Total cost: $0.2455
Total cost: $0.5672
SKIPPING: no scores found
Total cost: $0.8458
SKIPPING: scores must be one of ['N/A', 'Determinable - Certainly True', 'Determinable - True', 'Determinable - Uncertain', 'Determinable - False', 'Determinable - Certainly False'], but found ['N/A', 'Determinable - Certainly True', 'Determinable - Certainly True', 'Determinable - Certainly True', 'Determinable - Certain True', 'Determinable - Certainly True', 'Determinable - Certainly True', 'N/A']
Total cost: $1.1555
SKIPPING: 14 scores found, but 21 annotations were expected.
Total cost: $1.4406
Total cost: $1.6909
Total cost: $1.9656
Total cost: $2.2518
SKIPPING: 7 s

In [39]:
len(all_out_df)

19

In [40]:
# NOTE: THERE ARE DUPLICATE IDS IN THE OUTPUT DF WHEN USING RETRIEVER SO THAT WE CAN COMPARE THE RESULTS
# TODO: Change this once I've finished validating the retriever
out_df_retr = all_out_df[all_out_df["used_retrieval"]]
retr_ids = set(out_df_retr["message_id"])
out_df_counter_retr = all_out_df[all_out_df["message_id"].isin(retr_ids) & ~all_out_df["used_retrieval"]]  # counterfactually if retrieval was not used
# remove all the rows with message_id in retr_ids but with used_retrieval==False
out_df = all_out_df[~(all_out_df["message_id"].isin(retr_ids) & ~all_out_df["used_retrieval"])]

In [41]:
main_df.index = main_df["message_id"]
labeled_df = out_df.copy()
labeled_df.index = labeled_df["message_id"]
labeled_df = labeled_df.drop(columns=["message_id", "annotated_transcript"])
labeled_df = labeled_df.join(main_df, how="left")
print("length:", len(labeled_df))
# labeled_df.head()

length: 19


In [42]:
from datetime import datetime
# save results
dt = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
labeled_df.to_json(f"data/validation/dataframes/{dt}.json", orient="records")

In [43]:
# dt = "2023-09-13_20-22-18"
# labeled_df = pd.read_json(f"data/validation/dataframes/{dt}.json")
# labeled_df["scores"] = [val_df.loc[message_id]["scores"] for message_id in labeled_df["message_id"]]

In [44]:
import numpy as np
from statsmodels.stats.proportion import proportion_confint

sampled_df = labeled_df
cat_preds = np.concatenate(sampled_df["p_trues"].values)
cat_p_apt = np.concatenate(sampled_df["p_apts"].values)
cat_gt = np.concatenate(sampled_df["scores"].values)
n_statements = np.sum(sampled_df["ann_count"].values)
n_transcripts = len(sampled_df)

prop_retrieval = sampled_df["used_retrieval"].mean()

# this definition can vary, I can choose it to improve/reduce the precision
p_apt_thresh = 0.7
p_true_thresh = 0.99
p_false_thresh = 0.2
apt = cat_p_apt >= p_apt_thresh
pred_true = (cat_preds >= p_true_thresh) & apt
pred_false = (cat_preds <= p_false_thresh) & apt
pred_true_effective_n = pred_true.sum() # / n_statements * n_transcripts   # ultra conservative
pred_false_effective_n = pred_false.sum() # / n_statements * n_transcripts

prec_true = (cat_gt[pred_true] == "Certainly True").mean()
prec_false = (cat_gt[pred_false] == "Certainly False").mean()

anti_prec_true = (cat_gt[pred_true] == "Certainly False").mean()
anti_prec_false = (cat_gt[pred_false] == "Certainly True").mean()

prop_true = pred_true.sum() / n_statements
prop_false = pred_false.sum() / n_statements
proportion_apt = prop_true + prop_false

prop_gt_true = (cat_gt == "Certainly True").mean()
prop_gt_false = (cat_gt == "Certainly False").mean()

prec_confint = proportion_confint(prec_true * pred_true_effective_n, pred_true_effective_n, alpha=0.05, method="wilson")
print(f"Proportion of predicted true statements that are actually true: {prec_true:.3f} ({prec_confint[0]:.3f}, {prec_confint[1]:.3f})")
anti_prec_confint = proportion_confint(anti_prec_true * pred_true_effective_n, pred_true_effective_n, alpha=0.05, method="wilson")
print(f"Proportion of predicted true statements that are actually false: {anti_prec_true:.3f} ({anti_prec_confint[0]:.3f}, {anti_prec_confint[1]:.3f})")
prec_false_confint = proportion_confint(prec_false * pred_false_effective_n, pred_false_effective_n, alpha=0.05, method="wilson")
print(f"Proportion of predicted false statements that are actually false: {prec_false:.3f} ({prec_false_confint[0]:.3f}, {prec_false_confint[1]:.3f})")
anti_prec_false_confint = proportion_confint(anti_prec_false * pred_false_effective_n, pred_false_effective_n, alpha=0.05, method="wilson")
print(f"Proportion of predicted false statements that are actually true: {anti_prec_false:.3f} ({anti_prec_false_confint[0]:.3f}, {anti_prec_false_confint[1]:.3f})")
print(f"Proportion of statements that are actually true: {prop_gt_true:.3f}")
print(f"Proportion of statements that are actually false: {prop_gt_false:.3f}")
print(f"Proportion of statements that are actually true or false: {prop_gt_true + prop_gt_false:.3f}")
print(f"Proportion of statements that are predicted true: {prop_true:.3f}")
print(f"Proportion of statements that are predicted false: {prop_false:.3f}")
print(f"Proportion of statements that are predicted true or false: {prop_true + prop_false:.3f}")
print(f"Proportion of transcripts that use retrieval: {prop_retrieval:.3f}")
print(f"N transcripts: {n_transcripts}")
print(f"N statements: {n_statements}")

Proportion of predicted true statements that are actually true: 0.944 (0.819, 0.985)
Proportion of predicted true statements that are actually false: 0.000 (0.000, 0.096)
Proportion of predicted false statements that are actually false: 0.938 (0.717, 0.989)
Proportion of predicted false statements that are actually true: 0.000 (0.000, 0.194)
Proportion of statements that are actually true: 0.369
Proportion of statements that are actually false: 0.131
Proportion of statements that are actually true or false: 0.500
Proportion of statements that are predicted true: 0.182
Proportion of statements that are predicted false: 0.081
Proportion of statements that are predicted true or false: 0.263
Proportion of transcripts that use retrieval: 0.000
N transcripts: 19
N statements: 198


In [45]:
import json

with open("data/validation/logs.json", "r") as f:
    avg_scores = json.load(f)

# score_object = {
#     **truth_labeler.get_metadata(),
#     "p_apt_thresh": p_apt_thresh,
#     "p_true_thresh": p_true_thresh,
#     "proportion_apt": (np.mean(bootstrap_prop_apt), 2 * np.std(bootstrap_prop_apt)),
#     "datetime": dt,
#     "prec_true": (np.nanmean(bootstrap_prec_true), 2 * np.nanstd(bootstrap_prec_true)),
#     "prec_false": (np.nanmean(bootstrap_prec_false), 2 * np.nanstd(bootstrap_prec_false)),
#     "anti_prec_true": (np.nanmean(bootstrap_anti_prec_true), 2 * np.nanstd(bootstrap_anti_prec_true)),
#     "anti_prec_false": (np.nanmean(bootstrap_anti_prec_false), 2 * np.nanstd(bootstrap_anti_prec_false)),
#     "prop_retrieval": (np.mean(bootstrap_prop_retrieval), 2 * np.std(bootstrap_prop_retrieval)),
#     "prop_pred_true": (np.mean(bootstrap_prop_true), 2 * np.std(bootstrap_prop_true)),
#     "prop_pred_false": (np.mean(bootstrap_prop_false), 2 * np.std(bootstrap_prop_false)),
#     "prop_gt_true": (np.mean(bootstrap_prop_gt_true), 2 * np.std(bootstrap_prop_gt_true)),
#     "prop_gt_false": (np.mean(bootstrap_prop_gt_false), 2 * np.std(bootstrap_prop_gt_false)),
# }
score_object = {
    **truth_labeler.get_metadata(),
    "p_apt_thresh": p_apt_thresh,
    "p_true_thresh": p_true_thresh,
    "p_false_thresh": p_false_thresh,
    "proportion_apt": proportion_apt,
    "datetime": dt,
    "prec_true": prec_true,
    "prec_false": prec_false,
    "anti_prec_true": anti_prec_true,
    "anti_prec_false": anti_prec_false,
    "prop_retrieval": prop_retrieval,
    "prop_pred_true": prop_true,
    "prop_pred_false": prop_false,
    "prop_gt_true": prop_gt_true,
    "prop_gt_false": prop_gt_false,
    "n_statements": int(n_statements),
}
    
avg_scores.append(score_object)
with open("data/validation/logs.json", "w") as f:
    json.dump(avg_scores, f, indent=2)
score_object

{'model_name': 'gpt-4',
 'temperature': 1,
 'n_samples': 5,
 'uncertainty_bias': 2,
 'na_bias': 0,
 'score_to_p_apt': {'N/A': 0,
  'Determinable - Certainly True': 1,
  'Determinable - True': 1,
  'Determinable - Uncertain': 0.3,
  'Determinable - False': 1,
  'Determinable - Certainly False': 1},
 'score_to_p_true': {'N/A': 0.5,
  'Determinable - Certainly True': 1,
  'Determinable - True': 0.8,
  'Determinable - Uncertain': 0.5,
  'Determinable - False': 0.2,
  'Determinable - Certainly False': 0},
 'stop_seq': '[[TRANSCRIPT]] ',
 'total_cost': 2.65212,
 'system_prompt': 'You are an epistemically modest, and wise reviewer who is extraordinarily vigilant about getting facts right. You choose a good mix of options, and otherwise are well-calibrated, usually avoiding extreme options. You often say you are uncertain, and are careful and deliberate about when you say something is true.',
 'prompt_template': 'First, decide whether each marked statement can be given an accuracy label, writi

In [46]:
str = ""
for row in labeled_df.itertuples():
    statements = list(zip(row.p_trues, row.p_apts, row.scores))
    any_ = False
    for i, statement in enumerate(statements):
        if statement[2] != "Certainly True" and statement[0] >= p_true_thresh:
            print(i + 1)
            print("p_true: {}, p_apt: {}, gt: {}".format(*statement))
            str += f"{i + 1}\n"
            str += f"p_true: {statement[0]}, p_apt: {statement[1]}, gt: {statement[2]}\n"
            any_ = True
    if any_:
        print(row.annotated_assistant_text)
        print()
        str += f"{row.annotated_assistant_text}\n\n"
with open("data/tmp.txt", "w") as f:
    f.write(str)

14
p_true: 1.0, p_apt: 1.0, gt: True
Thank you for asking[[1]]! I'm here to help you with your question[[2]]. However, I must point out that the question contains some assumptions that may not be accurate[[3]]. cells taken from a body and grown in a nutritious and protected culture do not necessarily follow the same patterns of division and aging as they would inside a living organism[[4]]. The behavior of cells cultured outside of a body is complex and can vary depending on various factors such as the type of cell, the growth medium, and the conditions under which they are grown[[5]]. Therefore, it is important to approach this question with caution and avoid making assumptions based solely on what we might observe in natural tissues[[6]].
To provide an informed answer, let us consider each option carefully:[[7]]
(A) "Never stopping divisin": While it is true that many types of cells can continue to divide indefinitely when provided with the right conditions, this is not always the ca

In [15]:
str = ""
for row in labeled_df.itertuples():
    print(row.annotated_transcript)
    print(row.responses)
    str += f"{row.annotated_transcript}\n"
    str += f"{row.responses}\n"
    print(row.correct_answer)
    str += f"{row.correct_answer}\n"
    statements = list(zip(row.p_trues, row.p_apts, row.scores))
    for statement in statements:
        print("p_true: {}, p_apt: {}, gt: {}".format(*statement))
        str += "p_true: {}, p_apt: {}, gt: {}\n".format(*statement)
    print(row.pred_scores)
    str += f"{row.pred_scores}\n\n"
    print()
with open("data/tmp.txt", "w") as f:
    f.write(str)

USER: Answer this question:

What happens to the image formed by a concave mirror as the object is moved from far away to near the focal point?
(A) The image moves away from the mirror and gets shorter.
(B) The image moves away from the mirror and gets taller.
(C) The image moves toward the mirror and gets shorter.
(D) The image moves toward the mirror and gets taller.

ASSISTANT: Hello[[1]]! I'm here to help you with your question[[2]]. However, I must inform you that the statement in the question is incorrect[[3]]. When an object is moved from far away to near the focal point of a concave mirror, the image formed will actually appear larger, not smaller[[4]]. This is because the concave mirror has a negative focal length, which means it diverges light rays and makes objects appear larger than they really are[[5]]. So, the correct answer is (B) The image moves away from the mirror and gets taller.[[6]]
['[[BRIEF SUMMARY]] Because the correct answer is (B) The image moves away from the

In [10]:
# import numpy as np

# n_bootstraps = 2000
# bootstrap_prec_true = []
# bootstrap_prec_false = []
# bootstrap_anti_prec_true = []
# bootstrap_anti_prec_false = []
# bootstrap_prop_true = []
# bootstrap_prop_false = []
# bootstrap_prop_gt_true = []
# bootstrap_prop_gt_false = []
# bootstrap_prop_apt = []
# bootstrap_prop_retrieval = []
# bootstrap_n_statements = []
# for i in range(n_bootstraps):
#     sampled_df = labeled_df.sample(frac=1, replace=True)
#     cat_preds = np.concatenate(sampled_df["p_trues"].values)
#     cat_p_apt = np.concatenate(sampled_df["p_apts"].values)
#     cat_gt = np.concatenate(sampled_df["scores"].values)
#     expected_anns = np.sum(sampled_df["ann_count"].values)

#     prop_retrieval = sampled_df["used_retrieval"].mean()

#     # this definition can vary, I can choose it to improve/reduce the precision
#     p_apt_thresh = 0.7
#     p_true_thresh = 0.8
#     apt = cat_p_apt >= p_apt_thresh
#     pred_true = (cat_preds >= p_true_thresh) & apt
#     pred_false = (cat_preds <= (1 - p_true_thresh)) & apt

#     prec_true = (cat_gt[pred_true] == "Certainly True").mean()
#     prec_false = (cat_gt[pred_false] == "Certainly False").mean()

#     anti_prec_true = (cat_gt[pred_true] == "Certainly False").mean()
#     anti_prec_false = (cat_gt[pred_false] == "Certainly True").mean()

#     prop_true = pred_true.sum() / expected_anns
#     prop_false = pred_false.sum() / expected_anns
#     proportion_apt = prop_true + prop_false

#     prop_gt_true = (cat_gt == "Certainly True").mean()
#     prop_gt_false = (cat_gt == "Certainly False").mean()

#     bootstrap_prec_true.append(prec_true)
#     bootstrap_prec_false.append(prec_false)
#     bootstrap_anti_prec_true.append(anti_prec_true)
#     bootstrap_anti_prec_false.append(anti_prec_false)
#     bootstrap_prop_true.append(prop_true)
#     bootstrap_prop_false.append(prop_false)
#     bootstrap_prop_gt_true.append(prop_gt_true)
#     bootstrap_prop_gt_false.append(prop_gt_false)
#     bootstrap_prop_apt.append(proportion_apt)
#     bootstrap_prop_retrieval.append(prop_retrieval) 
#     bootstrap_n_statements.append(expected_anns)

# print(f"Proportion of predicted true statements that are actually true: {np.nanmean(bootstrap_prec_true):.3f} ± {2 * np.nanstd(bootstrap_prec_true):.3f}")
# print(f"Proportion of predicted true statements that are actually false: {np.nanmean(bootstrap_anti_prec_true):.3f} ± {2 * np.nanstd(bootstrap_anti_prec_true):.3f}")
# print(f"Proportion of predicted false statements that are actually false: {np.nanmean(bootstrap_prec_false):.3f} ± {2 * np.nanstd(bootstrap_prec_false):.3f}")
# print(f"Proportion of predicted false statements that are actually true: {np.nanmean(bootstrap_anti_prec_false):.3f} ± {2 * np.nanstd(bootstrap_anti_prec_false):.3f}")
# print(f"Proportion of statement predicted true: {np.mean(bootstrap_prop_true):.3f} ± {2 * np.std(bootstrap_prop_true):.3f}")
# print(f"Proportion of statement predicted false: {np.mean(bootstrap_prop_false):.3f} ± {2 * np.std(bootstrap_prop_false):.3f}")
# print(f"Proportion of statements that are labeled: {np.mean(bootstrap_prop_apt):.3f} ± {2 * np.std(bootstrap_prop_apt):.3f}")
# print(f"Proportion of statements that are true: {np.mean(bootstrap_prop_gt_true):.3f} ± {2 * np.std(bootstrap_prop_gt_true):.3f}")
# print(f"Proportion of statements that are false: {np.mean(bootstrap_prop_gt_false):.3f} ± {2 * np.std(bootstrap_prop_gt_false):.3f}")
# print(f"Proportion of transcripts using retrieval: {np.mean(bootstrap_prop_retrieval):.3f} ± {2 * np.std(bootstrap_prop_retrieval):.3f}")
# print(f"Number of statements: {np.mean(bootstrap_n_statements):.3f} ± {2 * np.std(bootstrap_n_statements):.3f}")


In [39]:
# TODO: test that the pipeline records accuracy correctly``
